In [13]:
from datetime import datetime, timedelta
from sqlalchemy import create_engine, MetaData, Table
from scrape_sofa import get_sofascore_api_data, init_driver, get_all_players_stats_fbref, get_all_teams_stats_fbref
from funciones import get_stat
import pandas as pd

today = datetime.now()
yesterday = today - timedelta(days=3)
yesterday = yesterday.strftime('%Y-%m-%d')
engine = create_engine('mysql+pymysql://root@localhost/footviz')
df_partidos_ayer = pd.read_sql(f"SELECT * FROM partido WHERE fecha = '{yesterday}'", engine)


In [2]:
driver =  init_driver()
data = get_sofascore_api_data(driver, path='api/v1/event/12436541/statistics')


In [11]:
info = data['statistics'][0]['groups'][0]['statisticsItems']

In [16]:
xg = get_stat(info, "Expected goals", default={"home":0, "away":0})
xg['home']

'1.27'